<a href="https://colab.research.google.com/github/AtulAravindDas/FinVAR_LLM_Version/blob/main/fmp_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-core langgraph>0.2.27

In [ ]:
!pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.8
    Uninstalling openai-1.99.8:
      Successfully uninstalled openai-1.99.8


In [ ]:
!pip install sec-edgar-downloader

In [ ]:
!pip install langchain-core langgraph>0.2.27
!pip install -U langchain-openai
!pip install sec-edgar-downloader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%writefile /content/drive/MyDrive/fmp_data_extract.py
import requests
import pandas as pd
import os
API_KEY = "QbXfO4JX704vVB7qu4fBGe5B6fiilEhn"
BASE_URL = "https://financialmodelingprep.com/api/v3"

class FMPDataExtractor:
    ticker_data_cache = {}

    def __init__(self, ticker):
        self.ticker = ticker.upper()
        self.api_key = API_KEY
        self.base_url = BASE_URL
    def compute_ratios(self, income_df, balance_df):
        ratios = {}
        try:
            year = income_df.columns.max()
            if len(income_df.columns) >= 2:
                rev_now = income_df.loc['revenue', income_df.columns[-1]]
                rev_prev = income_df.loc['revenue', income_df.columns[-2]]
                ratios['Revenue Growth (%)'] = ((rev_now - rev_prev) / rev_prev) * 100
        except:
            ratios['Revenue Growth (%)'] = None

        try:
            year = income_df.columns.max()
            current_assets = balance_df.loc['totalCurrentAssets', year]
            current_liabilities = balance_df.loc['totalCurrentLiabilities', year]
            ratios['Current Ratio'] = current_assets / current_liabilities
        except:
            ratios['Current Ratio'] = None

        try:
            year = income_df.columns.max()
            total_current_assets = balance_df.loc['totalCurrentAssets', year]
            total_current_liabilities = balance_df.loc['totalCurrentLiabilities', year]
            inventory= balance_df.loc['inventory', year]
            ratios['Quick Ratio'] = (total_current_assets - inventory) / total_current_liabilities
        except:
            ratios['Quick Ratio'] = None

        try:
            year = income_df.columns.max()
            total_debt = balance_df.loc['totalDebt', year]
            total_assets = balance_df.loc['totalAssets', year]
            ratios['Debt-to-Asset'] = total_debt / total_assets
        except:
            ratios['Debt-to-Asset'] = None

        try:
            year = income_df.columns.max()
            ebit = income_df.loc['ebit', year]
            interest_expense = income_df.loc['interestExpense', year]
            ratios['Interest Coverage Ratio'] = ebit / abs(interest_expense)
        except:
            ratios['Interest Coverage Ratio'] = None

        try:
          year = income_df.columns.max()
          ratios['FCF']=income_df.loc['netIncome', year]-income_df.loc['depreciationAndAmortization', year]
        except:
          ratios['FCF']=None


        return ratios


    def get_fmp_data(self):
        if self.ticker in FMPDataExtractor.ticker_data_cache and 'complete_data' in FMPDataExtractor.ticker_data_cache[self.ticker]:
            return FMPDataExtractor.ticker_data_cache[self.ticker]['complete_data']

        try:
            with requests.Session() as session:
                profile_url = f"{self.base_url}/profile/{self.ticker}?apikey={self.api_key}"
                quote_url = f"{self.base_url}/quote/{self.ticker}?apikey={self.api_key}"
                income_url = f"{self.base_url}/income-statement/{self.ticker}?limit=5&apikey={self.api_key}"
                balance_url = f"{self.base_url}/balance-sheet-statement/{self.ticker}?limit=5&apikey={self.api_key}"
                cashflow_url = f"{self.base_url}/cash-flow-statement/{self.ticker}?limit=5&apikey={self.api_key}"
                historical_url = f"{self.base_url}/historical-price-full/{self.ticker}?apikey={self.api_key}"

                profile_data = session.get(profile_url).json()
                quote_data = session.get(quote_url).json()
                income_data = session.get(income_url).json()
                balance_data = session.get(balance_url).json()
                cashflow_data = session.get(cashflow_url).json()
                historical_data = session.get(historical_url).json()

            if not profile_data or not quote_data:
                raise ValueError("Invalid data returned from FMP API.")

            profile = profile_data[0]
            quote = quote_data[0]

            info = {
                "longName": profile.get("companyName", "N/A"),
                "industry": profile.get("industry", "N/A"),
                "description": profile.get("description", "N/A"),
                "beta": profile.get("beta", None),
                "currentPrice": quote.get("price", None),
                "previousClose": quote.get("previousClose", None),
                "trailingPE": quote.get("pe", None),
                "epsTrailingTwelveMonths": quote.get("eps", None)
            }

            income_df = pd.DataFrame(income_data).set_index("date").T if income_data else pd.DataFrame()
            balance_df = pd.DataFrame(balance_data).set_index("date").T if balance_data else pd.DataFrame()
            cashflow_df = pd.DataFrame(cashflow_data).set_index("date").T if cashflow_data else pd.DataFrame()
            history_df = pd.DataFrame(historical_data.get('historical', [])) if 'historical' in historical_data else pd.DataFrame()

            if not history_df.empty:
                history_df['date'] = pd.to_datetime(history_df['date'])
                history_df.set_index('date', inplace=True)
                history_df = history_df.sort_index()

            if not income_df.empty:
                income_df.columns = pd.to_datetime(income_df.columns)
                income_df = income_df.apply(pd.to_numeric, errors='coerce')

            if not balance_df.empty:
                balance_df.columns = pd.to_datetime(balance_df.columns)
                balance_df = balance_df.apply(pd.to_numeric, errors='coerce')

            if not cashflow_df.empty:
                cashflow_df.columns = pd.to_datetime(cashflow_df.columns)
                cashflow_df = cashflow_df.apply(pd.to_numeric, errors='coerce')

            ratios = self.compute_ratios(income_df, balance_df)
            complete_data = {
                'info': info,
                'financials': (income_df, balance_df, cashflow_df, history_df),
                'ratios': ratios
            }

            FMPDataExtractor.ticker_data_cache[self.ticker] = {'complete_data': complete_data}
            return complete_data

        except Exception as e:
            return {
                'info': {"error": str(e)},
                'financials': (pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

            }

Overwriting /content/drive/MyDrive/fmp_data_extract.py
